In [1]:
import numpy as np
from itertools import accumulate
from gekko import GEKKO

Здесь нужно будет фигануть класс всего фреймворка

Изначально мы получаем на вход портфолио активов. Каждый актив характеризуется:
- количеством (при этом, если оно в минусе, то там позиции на продажу)
- глубина рынка
- волатильность
- дрифт
- лаг исполнения
- спотовая цена
- есть ли ежедневная переоценка

После получения данных мы определяем период ликвидации по той логике, чем быстрее тем лучше
Для разнообразия можно попробовать делать период 10 дней, если по формуле он выходит меньше

После этого генерируем ценовые пути на полученный временной интервал. Приписываем изначальные цены в начало этой трехмерной матрицы

Затем для этих ценовых путей определяем $\psi$ (psi). Она определяет потенциальный P&L для инструмента i при времени h в риск-сценарии $\mathbf{r_k}$, по факту из риск-сценария требуется только ценовой путь

Поскольку ценовых путей будет много, то выгодно сделать такие высоконагруженные операции в начале

Далее по идее начинается работа оптимизатора Branch\&Bound


#### Адаптация под оптимизаторы MINLP

Общие параметры

In [2]:
def portfolio_psi(portfolio, price_paths):
    quantities = np.array([i['quantity'] for i in portfolio]).reshape(len(portfolio), 1)
    ds = np.array([i['ds'] for i in portfolio]).reshape(len(portfolio),1)

    mid = price_paths[:, :, 1:] - ds * price_paths[:, :, :-1]
    return mid * quantities

In [3]:
# Parameters
H = 3

# задан портфель из двух инструментов
portfolio = [
    {
        'quantity': 1,
        'execution_lag': 3,
        'ds': False
    },
    {
        'quantity': -2,
        'execution_lag': 1,
        'ds': False
    }
]

In [4]:
price_path1 = np.array([[100, 130, 150, 160],
                        [100, 130, 150, 160]])

price_path2 = np.array([[100, 80, 70, 60],
                        [100, 80, 70, 60]])

price_path3 = np.array([[100, 110, 60, 80],
                        [100, 110, 60, 80]])

price_paths = np.array([price_path1,
                        price_path2,
                        price_path3])
price_paths.shape

(3, 2, 4)

# alternate case
price_paths = []
for _ in range(10_000):
    price_paths.append(np.random.rand(len(portfolio), H+1) * 200) # типо генерим сценарии

price_paths = np.array(price_paths)
price_paths.shape

In [5]:
# important variables
psi = portfolio_psi(portfolio, price_paths).tolist()
quantities = [i['quantity'] for i in portfolio]
ds = [i['ds'] for i in portfolio]
abs_quantities = [abs(i) for i in quantities]

psi

[[[130, 150, 160], [-260, -300, -320]],
 [[80, 70, 60], [-160, -140, -120]],
 [[110, 60, 80], [-220, -120, -160]]]

Целевая функция на чистом питоне

def pure_objective_func(portfolio, psi, strategy):
    cumsum = [list(accumulate(row)) for row in strategy]
    step11 = [[abs_quantities[i] - cumsum[i][h] for h in range(H)] for i in range(len(portfolio))]
    step12 = [[ds[i] * step11[i][h] for h in range(H)] for i in range(len(portfolio))]
    step13 = [[step12[i][h] + strategy[i][h] for h in range(H)] for i in range(len(portfolio))]

    step21 = [[[psi[k][i][h] * step13[i][h] for h in range(H)] for i in range(len(portfolio))] for k in range(len(psi))]
    step22 = [[[step21[k][i][h] / abs_quantities[i] for h in range(H)] for i in range(len(portfolio))] for k in range(len(psi))]

    step31 = [[sum(col) for col in zip(*m)] for m in step22]
    step32 = [list(accumulate(row)) for row in step31]

    # print(step32)

    first = [row[-1] for row in step32] # L(r_k, H)
    second = [min(row) for row in step32] # \min_{h} L(r_k, h)

    return min([i + j for i, j in zip(first, second)]) # WL case

In [6]:
def alt_pure_objective_func(portfolio, psi, strategy):
    cumsum = [list(accumulate(row)) for row in strategy]
    step1 = [[ds[i] * (abs_quantities[i] - cumsum[i][h]) + strategy[i][h] for h in range(H)] for i in range(len(portfolio))]

    step2 = [[[psi[k][i][h] * step1[i][h] / abs_quantities[i] for h in range(H)] for i in range(len(portfolio))] for k in range(len(psi))]

    step3 = [list(accumulate(map(sum, zip(*rows)))) for rows in step2]

    # print(step3)

    first = [row[-1] for row in step3] # L(r_k, H)
    second = [min(row) for row in step3] # \min_{h} L(r_k, h)

    return min([i + j for i, j in zip(first, second)]) # WL case

%%time
strat1 = [0, 0, 1]
strats2 = [[2, 0, 0],
           [1, 1, 0],
           [1, 0, 1],
           [0, 1, 1],
           [0, 2, 0],
           [0, 0, 2]]

for j in strats2:
    Q = [strat1, j]
    print(pure_objective_func(portfolio, psi, Q))

In [8]:
%%time
strat1 = [0, 0, 1]
strats2 = [[2, 0, 0],
           [1, 1, 0],
           [1, 0, 1],
           [0, 1, 1],
           [0, 2, 0],
           [0, 0, 2]]

for j in strats2:
    Q = [strat1, j]
    print(alt_pure_objective_func(portfolio, psi, Q))

-360.0
-400.0
-260.0
-300.0
-440.0
-320.0
CPU times: user 444 µs, sys: 270 µs, total: 714 µs
Wall time: 541 µs


In [9]:
# trying GEKKO
m = GEKKO()

#### Пробуем GEKKO

<img src="img2.png" alt="img2" width="500"/>

In [10]:
# define optimizable variable
# 2 условие Q_{i,h} >= 0
# 3 условие Q_{i,h} <= Q_{i,0}
strategy = [[m.Var(lb=0, ub=abs(i['quantity']), integer=True) for h in range(H)] for i in portfolio]
print(strategy)

[[0, 0, 0], [0, 0, 0]]


In [11]:
# 1 условие \sum_{h=1}^{H} Q_{i,h} = Q_{i,0}
for i in range(len(portfolio)):
    temp = 0
    for h in range(H):
        temp += strategy[i][h]

    m.Equation(portfolio[i]['quantity'] == temp)
    print(m.Equation(portfolio[i]['quantity'] == temp))

(((0+int_v1)+int_v2)+int_v3)=1
(((0+int_v4)+int_v5)+int_v6)=-2


In [12]:
# 5 условие
for i in range(len(portfolio)):
    for h in range(portfolio[i]['execution_lag'] - 1):
        m.Equation(strategy[i][h] == 0)
        print(m.Equation(strategy[i][h] == 0))

int_v1=0
int_v2=0


In [13]:
m.Obj(-alt_pure_objective_func(portfolio=portfolio,
                               psi=psi,
                               strategy=strategy))

TypeError: object of type 'int' has no len()

In [ ]:
m.options.SOLVER=1
m.solve()